#Image Classification using Convolutional Neural Networks

Rahul Nadkarni


In [ ]:
import tensorflow as tf
import os

Use the GPU to train the model faster

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: #Limit memory growth
  tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

Get the data files from google drive - Colab Notebook

In [ ]:
data_dir = '/content/drive/MyDrive/data'

Look for these files in the data folders

In [ ]:
image_exts = ['jpeg','jpg','bmp','png']

In [ ]:
image_exts[2]

'bmp'

Read the image

In [ ]:
img = cv2.imread(os.path.join('/content/drive/MyDrive/data','happy','smile.woman_.jpg'))

(length, width, dimensions(colors)) [link text](https://)

In [ ]:
img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

#Data Preprocessing

Convert the BGR file to RGB

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


Remove all image files that are not in the supported format list.

In [ ]:
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path = os.path.join(data_dir,image_class,image)
    try:
      img = cv2.imread(image)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print("Image not in ext list {}".format(image_path))
        os.remove(image_path)
    except Exception as e:
      print("Issue with image {}".format(image_path))

In [ ]:
tf.data.Dataset

In [ ]:
import numpy as np

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/data')

Train the model using batches

In [ ]:
data_iterator = data.as_numpy_iterator() #loop through it

In [ ]:
batch = data_iterator.next() #grab a batch - images and labels

In [ ]:
batch[1] #happy is 0, sad is 1

In [ ]:
fig,ax = plt.subplots(ncols = 4,figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
  ax[idx].imshow(img.astype(int))
  ax[idx].title.set_text(batch[1][idx])

Perform a scaled transformation on the data - convert the value of x to a value between 0 and 1.

In [ ]:
data = data.map(lambda x,y:(x/255,y))
#performs the scaled transformation in the pipeline

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
len(data)

Create Train, Test, Validation Splits






In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)* .2)
test_size = int(len(data)*.1) + 1

In [ ]:
train_size+val_size+test_size

In [ ]:
train = data.take(train_size) #how much data we take in this partition
val = data.skip(train_size).take(val_size) #skip the batches we already allocated and take the remaining for val size
test = data.skip(train_size + val_size).take(test_size) #everything left over


In [ ]:
len(test)

#Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential #functional for multiple inputs outputs -> not for this project
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten,Dropout

In [ ]:
model = Sequential()


In [ ]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape = (256,256,3)))
#16 filters 3x3 - stride of 1 pixel
#relu activation - pass output of convolution to relu
model.add(MaxPooling2D()) #take the max value and condense

model.add(Conv2D(32,(3,3),1,activation = 'relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),1,activation = 'relu'))

model.add(Flatten())
#flatten the matrix
#Fully Connected Layers
model.add(Dense(256,activation = 'relu')) # 256 neurons
model.add(Dense(1,activation = 'sigmoid')) #single dense layer and one output -> sigmoid

In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model,to_file='plot.png',show_layer_names = True)

In [ ]:
logdir = '/content/drive/MyDrive/logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)
#create a checkpoint

In [ ]:
tr = model.fit(train,epochs=20,validation_data=val,callbacks = [tensorboard_callback])

In [ ]:
tr.history

#Results

In [ ]:
fig = plt.figure()
plt.plot(tr.history['loss'],color ='blue',label = 'loss')
plt.plot(tr.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(tr.history['accuracy'],color ='blue',label = 'accuracy')
plt.plot(tr.history['val_accuracy'],color='orange',label='val_accuracy')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc='upper left')
plt.show()

#Test the Model

In [ ]:
import cv2

In [ ]:
img = cv2.imread('/content/drive/MyDrive/data/happy/Travis-Bradberry-Happy.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255,0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
  print(f'Predicted class is sad')
else:
  print(f'Predicted class is happy')